## Script to show the number of times the scratchpad was looked at in every trial

### Should be edited to compute the total timing of each look and if the sentence was correct or not

In [1]:
%matplotlib
#%matplotlib inline
import os
import csv
import fnmatch
import numpy as np
import datetime
import re
import matplotlib.pyplot as plt
from pathlib import Path
import pandas as pd


Using matplotlib backend: Qt5Agg


In [2]:
def maxPhrasesPossible(phrases):
    # function to find max stimulation phrases possible for the user
    
    phrasesOnly = [row[1] for row in phrases]
    if 'THE EXPERIMENT IS NOW DONE' in phrasesOnly:
        phrasesOnly.remove(phrasesOnly[-1])
        
    return len(phrasesOnly)

In [3]:
def FrequencyScratchPadPerTrial(UserLooksAtKey, maxPhraseNr):
    # function to find the frequency of looking at scratchpad per trial
    
    ScratchPadAtnPhrase = dict()
    nPhrase = 1 # assuming the first sentence is directly started, unless 'nextPhrase' is choosen
    nScratchPad = 0
    nRow = -1
    firstScratchPad = 0
    flagStart = 0
    
    for row in UserLooksAtKey:
        nRow = nRow + 1
        
        if row[1] == 'ScratchPad':
            
            # for consecutive scratchpad looks
            if UserLooksAtKey[nRow-1][1] == 'ScratchPad':
                if float(row[2]) == 0:
                    progressScratchPad = float(UserLooksAtKey[nRow-1][2])
                else:
                    progressScratchPad = float(row[2])
    
            # if first time of the look at scratchpad
            else:
                # if not the very first look (means the variable for progress exists)
                if firstScratchPad > 0: 
                    # if the current progress value is less than the previous one, means it is a new look 
                    if progressScratchPad > float(row[2]):
                        nScratchPad = nScratchPad + 1
                        ScratchPadAtnPhrase.update({nPhrase: nScratchPad})
                        # if progress is not 0, then update the progress --> sometimes, it is 0 at the end of the look
                        if float(row[2]) != 0:
                            progressScratchPad = float(row[2])
                            
                # if the very first look (means the variable for progress does not exist)    
                else:
                    nScratchPad = nScratchPad + 1
                    ScratchPadAtnPhrase.update({nPhrase: nScratchPad})                        
                    progressScratchPad = float(row[2])
                    firstScratchPad = 1
                    
        # if next phrase being typed                      
        if row[1] == 'NextPhrase':
            if float(row[2]) == 1: # if it is completely selected
                nPhrase = nPhrase + 1
                nScratchPad = 0 # reset to 0 for the next phrase 
                if nPhrase > maxPhraseNr:
                    break
                
    # for the looks after the experiment was completed, they are not counted
    
    return ScratchPadAtnPhrase
        

In [4]:
def AddDataToFile(metricComputed, Names, DataEvaluated):

    # Save the data
    fileIn = r'C:\DTU\Data\201805_HealthnRehab\data_summary.csv'

    if Path(fileIn).is_file():
        
        fSize = os.path.getsize(fileIn) # or pd.read_excel(filename) for xls file

        # file exists
        # check if the given column exists
        with open(fileIn, 'r', newline='') as csvfileRead:
            if fSize > 0:
                fileOut = fileIn[:-4] + '1.csv'
                with open(fileOut, 'w', newline='') as csvfileWrite:
                    for row in csv.reader(csvfileRead):
                        if metricComputed in row:
                            # file exists and the metric is present in the file
                            print('This metric is already calculated and saved: ', metricComputed)
                            break
                        else:
                            # file exists but the metric is not present
                            print('adding the metric')
                            filewriter = csv.writer(csvfileWrite, delimiter = ',')
                            if len(row) > 1:
                                rowJoined = list()
                                for rowElement in row:
                                    rowJoined.append(rowElement)
                                
                                
                                print(rowJoined[1])
                                
                                if rowJoined[1] == 'subject_name':
                                    # write the titles of the metrics
                                    rowJoined.append(metricComputed)
                                    filewriter.writerow(rowJoined)
                                else:
                                    #filewriter.writerow([row, str(DataEvaluated[Names.index(row[0])])])
                                    rowJoined.append(str(DataEvaluated[Names.index(row[1])]))
                                    filewriter.writerow(rowJoined)
                                
                            else:
                                if row[1] == 'subject_name':
                                    # write the titles of the metrics
                                    filewriter.writerow([row, metricComputed])
                                else:
                                    filewriter.writerow([row, str(DataEvaluated[Names.index(row[1])])])
            
            
            else:
                fileOut = fileIn
                print('recreating a file')

                with open(fileSave, 'w', newline='') as csvfileWrite:
                    filewriter = csv.writer(csvfileWrite, delimiter=',')
                    filewriter.writerow(['subject_name', metricComputed])
                    for ind in enumerate(Names):
                        filewriter.writerow([Names[ind], DataEvaluated[ind[0]]])
                
    else:
        print('creating a file')
        a = re.compile('(?<=TypingData)(.*)(?=OptiKey)')
        with open(fileSave, 'w', newline='') as csvfileWrite:
            filewriter = csv.writer(csvfileWrite, delimiter=',')
            filewriter.writerow(['subject_name', metricComputed])
            for ind in enumerate(Names):
                subjName = a.findall(Names[ind[0]])
                filewriter.writerow([subjName[0], DataEvaluated[ind[0]]])
     

In [5]:
metricComputed = 'read_text_events_frequency_per_trial_ratio'
dataFolderName = r'C:\DTU\Data\201805_HealthnRehab\TypingData'
j = 0
scratchPadToTypingRatioFrequencyList = list()
Names = list()

for root, dirs, subfolder in os.walk(dataFolderName):

    if not dirs:
        
        #if 'notCompleted' in root or 'notInclude' in root: # Some subjects do not have gaze log and have been marked as 
        
        if 'notCompleted' in root: # Some subjects do not have gaze log and have been marked as 
            #notInclude
            continue
        if 'tb' in root or 'joha' in root:
            continue
            
        scratchPad = None
        userKeys = None
        phraseLog = None
        
        for file in subfolder:
            
            if fnmatch.fnmatch(file, 'ScratchPadLog*'):
                try:
                    fScratchPad = open(root + '\\' + file, encoding='utf-8')
                    readerScratchPad = csv.reader(fScratchPad)
                    scratchPad = list(readerScratchPad)
                except:
                    if fScratchPad is not None:
                        fScratchPad.close()
                    else:
                        print('error in opening the scratchpad log file')
            
            if 'js' in root:
                if fnmatch.fnmatch(file, 'ScratchPadLog*'):
                    try:
                        fScratchPad = open(root + '\\' + file)
                        readerScratchPad = csv.reader(fScratchPad)
                        scratchPad = list(readerScratchPad)
                        scratchPad.remove(scratchPad[0])
                        #print(scratchPad)
                    except:
                        if fScratchPad is not None:
                            fScratchPad.close()
                        else:
                            print('error in opening the scratchpad log file')
            
            if fnmatch.fnmatch(file, 'user*'):
                try:
                    fUserKey = open(root + '\\' + file, encoding='utf-8')
                    readerUserKey = csv.reader(fUserKey)
                    userKeys = list(readerUserKey)
                    userKeys.remove(userKeys[0])
                except:
                    if fUserKey is not None:
                        fUserKey.close()
                    else:
                        print('error in opening the user key log file')
            
            if fnmatch.fnmatch(file, 'Phrase*'):
                try:
                    fStimPhraseKey = open(root + '\\' + file, encoding='utf-8')
                    readerStimPhraseKey = csv.reader(fStimPhraseKey)
                    stimPhrases = list(readerStimPhraseKey)
                    stimPhrases.remove(stimPhrases[0])
                except:
                    if fStimPhraseKey is not None:
                        fStimPhraseKey.close()
                    else:
                        print('error in opening the stim phrase log file')
            
        if scratchPad is None or userKeys is None or stimPhrases is None:
            continue
        else:
            
            print(root)
            
            # first count the number of sentences typed
            maxPhrases = maxPhrasesPossible(stimPhrases)
            
            # Number of times scratchpad was looked at
            ScratchPadAtnPhraseDict = FrequencyScratchPadPerTrial(userKeys, maxPhrases)
            print(ScratchPadAtnPhraseDict)
            
            # save only the name of the subject and the date part of the file
            a = re.compile('(?<=TypingData\\\May[0-9]{2}\\\)(.*)(?=\\\OptiKey)')
                
            Names.append(a.findall(root)[0])
            scratchPadToTypingRatioFrequencyList.append(ScratchPadAtnPhraseDict)
                
            #print(nScratchPad, nTypedChar)
                
            #nCount = nCount + 1
           
                
#if scratchPadToTypingRatioFrequencyList:
#    AddDataToFile(metricComputed, Names, scratchPadToTypingRatioFrequencyList)
                        

C:\DTU\Data\201805_HealthnRehab\TypingData\May15\akt_MS\OptiKeyLogs\2018-5-15-14-3-18
{1: 9, 2: 3, 4: 3, 5: 8}
C:\DTU\Data\201805_HealthnRehab\TypingData\May15\be_DT\OptiKeyLogs\2018-5-15-14-51-13-notInclude
{1: 2, 2: 6}
C:\DTU\Data\201805_HealthnRehab\TypingData\May15\jl_DT\OptiKeyLogs\2018-5-15-10-21-20
{2: 2}
C:\DTU\Data\201805_HealthnRehab\TypingData\May15\KEA_MS\OptiKeyLogs\2018-5-15-12-31-49
{1: 2, 2: 1, 4: 2, 5: 4}
C:\DTU\Data\201805_HealthnRehab\TypingData\May15\lone_DT\OptiKeyLogs\2018-5-15-11-4-33
{1: 11}
C:\DTU\Data\201805_HealthnRehab\TypingData\May15\MK_DT\OptiKeyLogs\2018-5-15-14-32-48
{1: 11, 2: 3, 3: 2}
C:\DTU\Data\201805_HealthnRehab\TypingData\May15\MT_MS\OptiKeyLogs\2018-5-15-10-52-23-notInclude
{1: 8, 2: 6, 3: 3, 4: 9}
C:\DTU\Data\201805_HealthnRehab\TypingData\May15\ok_MS\OptiKeyLogs\2018-5-15-12-23-16
{1: 6, 3: 2, 5: 2}
C:\DTU\Data\201805_HealthnRehab\TypingData\May15\pt_DT\OptiKeyLogs\2018-5-15-12-43-13
{1: 6, 2: 1, 4: 1, 5: 5}
C:\DTU\Data\201805_HealthnRehab\Typ

In [9]:
# Plot read text events for the trials for every user

fig = plt.figure()
ax = fig.add_subplot(1,1,1)
    
for readTextEvents in scratchPadToTypingRatioFrequencyList:
    x,y = zip(*sorted(readTextEvents.items())) # unpack a list of pairs into two tuples
    
    ax.plot(x, y)
    ax.set_xlabel('Trials')
    ax.set_ylabel('Frequency')
    ax.set_title('Frequency of looking at scratchpad per trial')

In [ ]:
# average number of times looked 